In [1]:
\c 5 100
h:`$":tpr-mscf-kx.tepper.cmu.edu:5000:mscf2023:LZ8AByXHDaEJdTim"

In [2]:
/%python
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from scipy.optimize import minimize
from scipy.optimize import OptimizeResult

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

plt.style.use('default')

In [3]:
/%python 

#you may want to install tensorflow if you don't have it currently (uncomment following line and run cell)
#!pip install tensorflow 

In [4]:
/ since we only have ~30 days of data, we use ~50:50 split
train_test_split_date: 2020.02.15 

hr: 7; 1
minute: 29; 1
sec: 29; 1

1


1


1


# Load Data

In [15]:
loaddata:{[s;c;sd;ed]
 / YOUR CODE HERE
 t:select date,sym,time,price,size from trade where date within (sd;ed), sym in s, cond in c;
 t}

show t_save:h (loaddata;`AAPL`NVDA`BABA;" ";2020.02.01;2020.03.01)

date       sym  time                 price  size
------------------------------------------------
2020.02.03 AAPL 0D09:30:00.104222076 304.25 200 
2020.02.03 AAPL 0D09:30:00.104222222 304.25 300 
2020.02.03 AAPL 0D09:30:00.115302599 304.16 100 
2020.02.03 AAPL 0D09:30:00.420630361 304.18 120 
2020.02.03 AAPL 0D09:30:00.425902385 304.18 100 
..


In [16]:
train_data: select from t_save where date < train_test_split_date
test_data: select from t_save where date >= train_test_split_date

In [17]:
count train_data

1409243


# Calculate predictability for k stocks
1. For seconds, minutes, hours frequency over D days, get MSE_i for stock i
2. return (k, n/D, 3)
3. Prediction model (average)
   - Autoregressive moving average model (ARIMA)
   - LSTM

In [20]:
get_features:{[t]
 // return the average price and sum of trading sizes within the given time step
 x_sec: `sym`date`second xasc select price:(sum (price*size))%sum size,size:sum size by date, time.second, sym from t;
 x_min: `sym`date`minute xasc select price:(sum (price*size))%sum size,size:sum size by date, time.minute, sym from t;
 x_hr: `sym`date`hh xasc select price:(sum (price*size))%sum size,size:sum size by date, time.hh, sym from t;
 dfs: (x_sec;x_min;x_hr);
 dfs}

dfs_train: get_features[train_data]
x_sec_train: dfs_train[0]
x_min_train: dfs_train[1]
x_hr_train: dfs_train[2]

dfs_test: get_features[test_data]
x_sec_test: dfs_test[0]
x_min_test: dfs_test[1]
x_hr_test: dfs_test[2]

In [21]:
count x_min_train

11713


In [22]:
x_sec_train

date       second   sym | price    size 
------------------------| --------------
2020.02.03 09:30:00 AAPL| 304.1988 1605 
2020.02.03 09:30:01 AAPL| 303.8334 25192
2020.02.03 09:30:02 AAPL| 303.5746 42265
2020.02.03 09:30:03 AAPL| 303.3663 46598
2020.02.03 09:30:04 AAPL| 302.7891 76661
..


In [23]:
tab2df:{r:.p.import[`pandas; `:DataFrame; x][@; cols x];$[count k:keys x; r[`:set_index]k; r]}
.p.set[`x_sec_train] tab2df x_sec_train
.p.set[`x_min_train] tab2df x_min_train
.p.set[`x_hr_train] tab2df x_hr_train

.p.set[`x_sec_test] tab2df x_sec_test
.p.set[`x_min_test] tab2df x_min_test
.p.set[`x_hr_test] tab2df x_hr_test

In [24]:
/%python
x_sec_train = x_sec_train.reset_index()
x_min_train = x_min_train.reset_index()
x_hr_train = x_hr_train.reset_index()

/%python
x_sec_test = x_sec_test.reset_index()
x_min_test = x_min_test.reset_index()
x_hr_test = x_hr_test.reset_index()

In [25]:
/%python
print(x_min_train)

       date  minute   sym       price    size
0      7338     570  AAPL  303.119055  763455
1      7338     571  AAPL  303.366303  336694
2      7338     572  AAPL  304.556376  312660
3      7338     573  AAPL  305.350792  297707
4      7338     574  AAPL  306.152994  256220
...     ...     ...   ...         ...     ...
11708  7349     956  NVDA  290.312920   36544
11709  7349     957  NVDA  290.191727   40705
11710  7349     958  NVDA  290.074701   67262
11711  7349     959  NVDA  290.025955   86985
11712  7349     960  NVDA  289.800000    5777

[11713 rows x 5 columns]


In [26]:
/%python
def convert_gregorian_date(date, input, scale):
    hour, minute, second = 0, 0, 0
    if scale == 'hour':
        hour = input
    elif scale == 'minute':
        hour = input // 60
        minute = input % 60
    else:
        second = input % 60
        minute = (input//60) % 60
        hour = (input//60//60) % 60
    mjd_start = datetime.datetime(2000, 1, 1)
    gregorian_date = mjd_start + datetime.timedelta(date)
    
    # Combine with the hour
    datetime_obj = datetime.datetime(gregorian_date.year, gregorian_date.month, gregorian_date.day, hour, minute, second)
    
    # Convert to Pandas datetime
    pandas_datetime = pd.to_datetime(datetime_obj)

    return pandas_datetime

x_sec_train['date'] = x_sec_train.apply(lambda row : convert_gregorian_date(row['date'], row['second'], scale='second'), axis=1)
x_sec_train.drop(['second'], axis=1, inplace=True)
x_sec_train.set_index('date', inplace=True)

x_min_train['date'] = x_min_train.apply(lambda row : convert_gregorian_date(row['date'], row['minute'], scale='minute'), axis=1)
x_min_train.drop(['minute'], axis=1, inplace=True)
x_min_train.set_index('date', inplace=True)

x_hr_train['date'] = x_hr_train.apply(lambda row : convert_gregorian_date(row['date'], row['hh'], scale='hour'), axis=1)
x_hr_train.drop(['hh'], axis=1, inplace=True)
x_hr_train.set_index('date', inplace=True)

In [39]:
/%python
x_sec_test['date'] = x_sec_test.apply(lambda row : convert_gregorian_date(row['date'], row['second'], scale='second'), axis=1)
x_sec_test.drop(['second'], axis=1, inplace=True)
x_sec_test.set_index('date', inplace=True)

x_min_test['date'] = x_min_test.apply(lambda row : convert_gregorian_date(row['date'], row['minute'], scale='minute'), axis=1)
x_min_test.drop(['minute'], axis=1, inplace=True)
x_min_test.set_index('date', inplace=True)

x_hr_test['date'] = x_hr_test.apply(lambda row : convert_gregorian_date(row['date'], row['hh'], scale='hour'), axis=1)
x_hr_test.drop(['hh'], axis=1, inplace=True)
x_hr_test.set_index('date', inplace=True)

In [27]:
/%python
print(x_min_train)

                      sym       price    size
date                                         
2020-02-03 09:30:00  AAPL  303.119055  763455
2020-02-03 09:31:00  AAPL  303.366303  336694
2020-02-03 09:32:00  AAPL  304.556376  312660
2020-02-03 09:33:00  AAPL  305.350792  297707
2020-02-03 09:34:00  AAPL  306.152994  256220
...                   ...         ...     ...
2020-02-14 15:56:00  NVDA  290.312920   36544
2020-02-14 15:57:00  NVDA  290.191727   40705
2020-02-14 15:58:00  NVDA  290.074701   67262
2020-02-14 15:59:00  NVDA  290.025955   86985
2020-02-14 16:00:00  NVDA  289.800000    5777

[11713 rows x 3 columns]


## LSTM

prepare data for tensorflow lstm

In [29]:
/%python
min_seq_length = 29
#train 
X_train, X_test, y_train, y_test = [], [], [], []
for sym in x_min_train.sym.unique():
    tmp = x_min_train[x_min_train['sym'] == sym].copy().reset_index()
    scaler = MinMaxScaler()
    tmp['price'] = scaler.fit_transform(tmp[['price']])

    for i in range(len(tmp) - min_seq_length - 1):
        X_train.append(tmp['price'].values[i:i+min_seq_length])
        y_train.append(tmp['price'].values[i+min_seq_length])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [30]:
/%python
print(X_train.shape, y_train.shape)

(11623, 29) (11623,)


In [34]:
/%python
model = Sequential()
# 64 is dimension of the model 
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], 1)))
# dense 1 for one-step ahead prediction 
model.add(Dense(1)) 
model.compile(optimizer='adam', loss='mse')

In [35]:
/%python
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
364/364 [==============================] - 2s 4ms/step - loss: 0.0088
Epoch 2/5
364/364 [==============================] - 1s 4ms/step - loss: 3.6074e-04
Epoch 3/5
364/364 [==============================] - 1s 4ms/step - loss: 2.6808e-04
Epoch 4/5
364/364 [==============================] - 1s 4ms/step - loss: 2.3962e-04
Epoch 5/5
364/364 [==============================] - 1s 4ms/step - loss: 2.1099e-04


In [48]:
/%python
print(x_min_test.index.date == np.unique(x_min_test.index.date)[0])

[ True  True  True ... False False False]


In [71]:
/%python
min_seq_length = 29
#test
columns = ['sym', 'date', 'mse']
result = pd.DataFrame(columns=columns)
def g(x):
    return x.strftime('%Y-%m-%d')

for sym in x_min_test.sym.unique():
    for date in sorted(np.unique(x_min_test.index.date)):
        X_test, y_test = [], []
        tmp = x_min_test[x_min_test['sym'] == sym].copy()
        tmp['dates'] = tmp.index.date
        tmp['dates'] = tmp['dates'].apply(g)
    
        tmp = tmp[tmp['dates'] == str(date)]
        
        
        scaler = MinMaxScaler()
        tmp['price'] = scaler.fit_transform(tmp[['price']])

        for i in range(len(tmp) - min_seq_length - 1):
            X_test.append(tmp['price'].values[i:i+min_seq_length])
            y_test.append(tmp['price'].values[i+min_seq_length])

        X_test = np.array(X_test)
        y_test = np.array(y_test)

        predictions = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions)
        y_test = scaler.inverse_transform(np.array([[i] for i in y_test]))

        predictions = np.array([i[0] for i in predictions])
        y_test = np.array([i[0] for i in y_test])

        mse = np.mean(sum((predictions - y_test)**2))   

        data_to_append = {'sym': sym, 'date': date, 'mse':mse}
        new_data = [data_to_append]
        new_df = pd.DataFrame(new_data, columns=columns)
        result = pd.concat([result, new_df], ignore_index=True)

12/12 [==============================] - 0s 1ms/step


In [72]:
/%python
result['predictability'] = 10/result['mse']
print(result)

     sym        date         mse  predictability
0   AAPL  2020-02-18    9.540642        1.048147
1   AAPL  2020-02-19    4.569171        2.188581
2   AAPL  2020-02-20   16.955913        0.589765
3   AAPL  2020-02-21   17.626297        0.567334
4   AAPL  2020-02-24   34.572656        0.289246
5   AAPL  2020-02-25   63.166462        0.158312
6   AAPL  2020-02-26   58.466220        0.171039
7   AAPL  2020-02-27  119.740932        0.083514
8   AAPL  2020-02-28  251.474519        0.039765
9   BABA  2020-02-18    4.153271        2.407741
10  BABA  2020-02-19    3.003813        3.329102
11  BABA  2020-02-20   20.046818        0.498832
12  BABA  2020-02-21    9.273950        1.078289
13  BABA  2020-02-24   12.520254        0.798706
14  BABA  2020-02-25   16.447196        0.608006
15  BABA  2020-02-26   22.087120        0.452753
16  BABA  2020-02-27   41.219737        0.242602
17  BABA  2020-02-28   41.505449        0.240932
18  NVDA  2020-02-18   27.491416        0.363750
19  NVDA  2020-02-19

# Liquidity

In [73]:
liquidity:{[s;sd;ed]
 / YOUR CODE HERE
 t:select date, sym, close, price, size from daily where date within (sd;ed), sym in s;
 t:update illiquidity: abs[log close%prev close]%price by sym from t;
 t}

In [75]:
show liquidity_df: h (liquidity; `AAPL`MRNA; 2020.02.17; 2020.02.28)

date       sym  close  price        size     illiquidity 
---------------------------------------------------------
2020.02.18 AAPL 319.75 1.308471e+10 41227703             
2020.02.18 MRNA 19.08  8.148386e+07 4321874              
2020.02.19 AAPL 324.22 8.520131e+09 26363364 1.629418e-12
2020.02.19 MRNA 19.15  6.363299e+07 3354162  5.754955e-11
2020.02.20 AAPL 319.2  8.544906e+09 26611284 1.826168e-12
..


In [76]:
.p.set[`liquidity_df] tab2df liquidity_df

In [77]:
/%python 
def f(x):
    #convert date time into readable form 
    original_date_str = "2020-02-03"
    original_date = datetime.datetime.strptime(original_date_str, "%Y-%m-%d")
    new_date = original_date + datetime.timedelta(days=x - 7339)
    new_date_str = new_date.strftime("%Y-%m-%d")
    return new_date_str
liquidity_df['date'] = liquidity_df['date'].apply(f)

In [79]:
/%python 
liquidity_df['liquidity']= 1/liquidity_df['illiquidity'] * 1e-9
print(liquidity_df)

          date   sym     close  ...       size   illiquidity    liquidity
0   2020-02-17  AAPL  319.7500  ...   41227703           NaN          NaN
1   2020-02-17  MRNA   19.0800  ...    4321874           NaN          NaN
2   2020-02-18  AAPL  324.2200  ...   26363364  1.629418e-12   613.715938
3   2020-02-18  MRNA   19.1500  ...    3354162  5.754955e-11    17.376332
4   2020-02-19  AAPL  319.2000  ...   26611284  1.826168e-12   547.594788
5   2020-02-19  MRNA   18.6000  ...    5102817  3.052522e-10     3.275980
6   2020-02-23  AAPL  301.1000  ...   61251363  3.183945e-12   314.075761
7   2020-02-23  MRNA   22.0000  ...    7889616  1.098744e-09     0.910130
8   2020-02-24  AAPL  288.3900  ...   60986813  2.413038e-12   414.415318
9   2020-02-24  MRNA   25.9700  ...   66516551  1.085318e-10     9.213890
10  2020-02-25  AAPL  290.2999  ...   52675770  4.280385e-13  2336.238287
11  2020-02-25  MRNA   32.9800  ...   79764455  1.049101e-10     9.531972
12  2020-02-26  AAPL  270.8000  ...   

# Sample Comparison